In [ ]:
# WORK IN PROGRESS!

# smemp_keck_and_apf.ipynb
# The current methods for reading in and operating on a spectrum used by Specmatch-emp assume the spectrum is in the format 
# produced by Keck/HIRES. Although there exists a method to read in APF spectra, it does not include certain pre-processing 
# such as combing spectra of the same star and deblazing) that the user may want to perform before Specmatch-emp is called. 
# This script allows the user to run Specmatch on both Keck and APF spectra.
# Use smemp_multifile.ipynb forjust APF spectra.
# Use code from the SpecMatch-Emp Quickstart Tutorial (https://specmatch-emp.readthedocs.io/en/latest/quickstart.html) for
# just Keck spectra.
# Last modified 8/12/20 by Anna Zuckerman

# THIS SCRIPT HAS NOT YET BEEN FULLY TESTED. IT RUNS WITHOUT ERROR BUT IS NOT GUARANTEED TO PRODUCE THE DESIRED RESULT.
# UPDATE: bugs currently in this script. Warning in deblaze for APF spectrum that didn't get warning in smemp_multifile,
#         and errors out in match() when running Keck spectra.

In [2]:
import numpy as np
import import_ipynb 
import pandas as pd 
import specmatchemp.library
import specmatchemp.plots as smplot
from specmatchemp.spectrum import Spectrum
from specmatchemp import spectrum
from specmatchemp.specmatch import SpecMatch
import astropy.io.fits as pf
from rescale import get_rescaled_wave_soln
from rescale import resample
from optparse import OptionParser
from bstar_deblaze import bstar_deblazed2 
from os import listdir
from os.path import isfile, join

importing Jupyter notebook from rescale.ipynb
importing Jupyter notebook from bstar_deblaze.ipynb


In [3]:
#Input: path to spectra/um corresponding to (ideally one night of observations of) a single star. Single fits file or directory.
#       If mulitple nights, may have to register each set idividually. 
#Output: Spectrum object for input star (see Specmatch-emp documentation)
#NOTE: There is a method called read_apf_fits that is already part of Specmatch-emp. However,it reads directly from a 
#      a fits file into a Spectrum object without the pre-processing done by this method.
def get_apf_spectrum(path_name):
        # get list of fits files
        try:
            filenames = [f for f in listdir(path_name) if isfile(join(path_name, f))]
        except NotADirectoryError: # path to one file
            path_split = path_name.split('/')
            path_split[:-1] = ['/'.join(path_split[:-1])]
            filenames = []
            filename = path_split[-1]
            filenames.append(filename)
            path_name = path_split[0]
        
        # Read in data from wavelength solution
        wave_soln = (pf.open('apf_wav.fits'))[0].data

        # Sum all of the data files for a star
        data = np.zeros((79, 4608))
        ve = False
        for filename in filenames:
            file = pf.open(path_name + '/' + filename)
            data_part = file[0].data
            if (str(np.shape(data_part)) != '(79, 4608)'):
                print(str(np.shape(data_part)) + ' is an incompatible data shape.')
                print('Cannot perform shift-and-match process.')
                sys.exit()
            try:
                data += data_part
            except ValueError:
                ve = True
        if (ve):
            print("Value Error occurred during spectra summation.")

        header = file[0].header # assumes all names are the same over all files
        name = header['TOBJECT']

        #Deblaze the orders: 31 to 52 (range of focus in the SME library)
        ve = False
        for order_inc in range(22):
            try: 
                data[30 + order_inc, :4600] = bstar_deblazed2(data,30 + order_inc)
            except ValueError: ve = True
        if (ve): print("Value Error occurred during blaze correction.")
            
        # option to remove cosmic rays. Use for calibration purposes only, as may remove real signals of interest  
        remove_cosmic_rays = False
        def remove_cosmic_rays(spect): # must input a normalized, deblazed spectrum
            new_spect = spect 
            for i in range(len(spect)):
                old_value = spect[i]
                if old_value > 1.4:
                    new_value = np.median(spect[i-3:i+3])
                    new_spect[i] = new_value # Other way would be to make this spect[i] not new_spect[i], then return spect
                    print('replaced value ' + str(old_value) + ' with '+ str(new_value) +' at ' + str(i))
            return new_spect
        if remove_cosmic_rays == True:
            data = remove_cosmic_rays(data)  
            
        # Get a wavelength solution rescaled onto the scale of the library
        wave_soln_ref = get_rescaled_wave_soln()

        # Resample the spectrum onto the new wavelength scale
        data_new = resample(wave_soln_ref, wave_soln, data)

        # Create Spectrum object
        apf_spectrum = Spectrum(np.asarray(wave_soln_ref), np.asarray(data_new))
        apf_spectrum.name = name
        
        return apf_spectrum

In [4]:
#Input: path to spectrum corresponding to a single star. Single fits file.
#Output: Spectrum object for input star (see Specmatch-emp documentation)
def get_keck_spectrum(pathname):
    hires_spectrum = spectrum.read_hires_fits(pathname).cut(5130,5210)
    return hires_spectrum

In [ ]:
# example keck spectum HD190406 at ./specmatch-emp-mod/samples/rj130.2075.fits

pathname = input('Enter path: ')
obs_type = input('Enter 0 for Keck files, 1 for APF files : ')
in_lib = input('Is the spectrum in the library? (0 -> no, 1 -> yes): ')
if int(obs_type) == 0:
    spect = get_keck_spectrum(pathname)
elif int(obs_type) == 1:
    spect = get_apf_spectrum(pathname)
else:
    print('Please enter 0 or 1.')
        
# below here, use any methods of Specmatch-emp
lib = specmatchemp.library.read_hdf()

# remove star from library if running on a spectrum in the library
if int(in_lib) == 1:
    lib_name = input('Enter star name as listed in Specmatch library: ')
    idx = lib.get_index(lib_name)
    star = lib.pop(idx)
    
sm = SpecMatch(spect, lib)

# Perform shift
sm.shift()

# Perform match
sm.match()

# Perform lincomb
sm.lincomb()
  